In [19]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from pathlib import Path

In [20]:
path = '../../prepare_data/data/base_line.csv'
df = pd.read_csv(Path(path).resolve())
df.drop('match_id', axis=1, inplace=True)
df.head()

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
0,23,2023,Sheffield United,Aston Villa,Bramall Lane,0
1,23,2023,Brighton & Hove Albion,Crystal Palace,American Express Stadium,1
2,23,2023,Burnley,Fulham,Turf Moor,2
3,23,2023,Newcastle United,Luton Town,St. James' Park,2
4,23,2023,Everton,Tottenham Hotspur,Goodison Park,2


In [21]:
train = df.iloc[400:]
val = df.iloc[:400]

In [22]:
train

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
400,22,2022,Manchester United,Crystal Palace,Old Trafford,1
401,22,2022,Wolverhampton Wanderers,Liverpool,Molineux Stadium,1
402,22,2022,Everton,Arsenal,Goodison Park,1
403,22,2022,Chelsea,Fulham,Stamford Bridge,2
404,21,2022,Fulham,Tottenham Hotspur,Craven Cottage,0
...,...,...,...,...,...,...
3638,1,2014,Queens Park Rangers,Hull City,Loftus Road,0
3639,1,2014,Stoke City,Aston Villa,bet365 Stadium,0
3640,1,2014,West Bromwich Albion,Sunderland,The Hawthorns,2
3641,1,2014,West Ham United,Tottenham Hotspur,Boleyn Ground,0


In [23]:
y_train = train['team_1_hue']
x_train = train.drop('team_1_hue', axis=1)

y_val = val['team_1_hue']
x_val = val.drop('team_1_hue', axis=1)

In [24]:
model_baseline = CatBoostClassifier()

In [25]:
model_baseline.fit(x_train, y_train, cat_features=list(x_train.columns), verbose=False)

In [26]:
pred = model_baseline.predict(x_val)
pred_proba = model_baseline.predict_proba(x_val)
y_val = y_val.to_numpy().reshape(-1, 1)

In [27]:
pred = np.concatenate((y_val, pred, pred_proba), axis=1)
df_pred = pd.DataFrame(pred, columns=['true', 'pred', '0_prob', '1_prob', '2_prob'])

In [28]:
df_pred['error'] = np.where(df_pred['true']==df_pred['pred'], 0, 1)
df_pred.head()


,true,pred,0_prob,1_prob,2_prob,error
0,0.0,1.0,0.406367,0.446242,0.147391,1
1,1.0,1.0,0.181168,0.740292,0.078540,0
2,2.0,1.0,0.224846,0.455246,0.319908,1
3,2.0,1.0,0.067131,0.688972,0.243897,1
4,2.0,1.0,0.301526,0.517391,0.181083,1


In [29]:
df_pred['error'].sum()

212

In [30]:
accuracy_score(df_pred['true'], df_pred['pred'])

0.47

In [31]:
f1_score(df_pred['true'], df_pred['pred'], average='weighted')

0.44102234237982435